# IMPORTS AND API_KEY

In [7]:
import requests
import time
import mysql.connector
import config
api_key = '91011d12ab785b50295a8f31e8f78a6b'
db_name = 'movies'

cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = 'movies'
)
cursor = cnx.cursor(buffered=True)

# CREATE DATABASE

# CREATE TABLE(S)

"create_table" creates tables into your database. Takes a query and database as variables.

In [9]:
def create_tmdb_table(table_name, db_name):
    cursor.execute("""
    CREATE TABLE {} (
    movie_id varchar(20),
    imdb_id varchar(20),
    title varchar(100) NOT NULL,
    budget int(12),
    revenue int(12),
    roi float(20),
    release_date varchar(10),
    PRIMARY KEY (movie_id)
    )
""".format(table_name))

In [10]:
create_tmdb_table('tmdb', db_name)

# DEFINE FUNCTIONS

"get_movie_ids" makes API calls to pull movie_ids (needed for reveneue/budget API calls). Takes a genre_id as a variable. Uses api_key defined above. Run time is approximately 30 seconds.

In [11]:
def get_movie_ids(genre_id):
    movies = []
    page = 1
    while page <= 100:
        url = 'https://api.themoviedb.org/3/discover/movie?api_key={}&with_genres={}&sort_by=revenue.desc&page={}'.format(api_key, genre_id, page)
        response = requests.get(url)
        moviez = response.json()['results']
        movies.extend(moviez)
        page += 1
        time.sleep(0.25)
        
    movie_ids = [movie['id'] for movie in movies]
    print(f"Got movie_ids. Now calling revenue and budget information for {len(movie_ids)} movies.")
    return movie_ids

"get_RevBub" makes one API call per movie_id and grabs revenue and budget infomation. Takes a movie_id as a variable. Uses api_key defined above.

In [12]:
def get_RevBud(i_d):
    url = 'https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US&append_to_response=revenue'.format(i_d,api_key)
    response = requests.get(url)
    movie = response.json()
    time.sleep(0.3)
    return movie

"parsed_results" returns cleaned results: imdb_id, title, budget, revenue, roi, release_date. Takes a list of movies (one dictionary per movie) as a variable.

In [13]:
def parsed_result(movie):
    parsed_movie = {}
    parsed_movie['movie_id'] = movie['id']
    parsed_movie['imdb_id'] = movie['imdb_id']
    parsed_movie['title'] = movie['title']
    parsed_movie['budget'] = movie['budget']
    parsed_movie['revenue'] = movie['revenue']
    parsed_movie['release_date'] = movie['release_date']
    return parsed_movie

"calc_roi" returns ROI = revenue / budget - 1. Takes a list of movies (one dictionary per movie) as a variable.

In [14]:
def calc_roi(movie):    
    if movie['budget'] != 0:
        movie['roi'] = round(((movie['revenue'] / movie['budget'])-1)*100, 2)
    else:
        movie['roi'] = 0.0
    return movie

In [15]:
def insert_into_DB(table_name, movie):
    movie_to_insert = (
        movie['movie_id'],
        movie['imdb_id'],
        movie['title'],
        movie['budget'],
        movie['revenue'],
        movie['roi'],
        movie['release_date']
        )
        
    insert_query = ("""
        INSERT INTO {} (movie_id, imdb_id, title, budget, revenue, roi, release_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """.format(table_name))
    
    cursor.execute(insert_query, movie_to_insert)
    cnx.commit()

In [16]:
def execute(genre_id, table_name):
    movie_ids = get_movie_ids(str(genre_id))
    for index, i_d in enumerate(movie_ids):
        index += 1
        movie = get_RevBud(i_d)
        parsed_movie = parsed_result(movie)
        movie_plus_roi = calc_roi(parsed_movie)
        insert_into_DB(table_name, movie_plus_roi)
        print(f"Inserted {index} movies into {table_name}")

In [17]:
execute(27, 'tmdb')

Got movie_ids. Now calling revenue and budget information for 2000 movies.
Inserted 1 movies into tmdb
Inserted 2 movies into tmdb
Inserted 3 movies into tmdb
Inserted 4 movies into tmdb
Inserted 5 movies into tmdb
Inserted 6 movies into tmdb
Inserted 7 movies into tmdb
Inserted 8 movies into tmdb
Inserted 9 movies into tmdb
Inserted 10 movies into tmdb
Inserted 11 movies into tmdb
Inserted 12 movies into tmdb
Inserted 13 movies into tmdb
Inserted 14 movies into tmdb
Inserted 15 movies into tmdb
Inserted 16 movies into tmdb
Inserted 17 movies into tmdb
Inserted 18 movies into tmdb
Inserted 19 movies into tmdb
Inserted 20 movies into tmdb
Inserted 21 movies into tmdb
Inserted 22 movies into tmdb
Inserted 23 movies into tmdb
Inserted 24 movies into tmdb
Inserted 25 movies into tmdb
Inserted 26 movies into tmdb
Inserted 27 movies into tmdb
Inserted 28 movies into tmdb
Inserted 29 movies into tmdb
Inserted 30 movies into tmdb
Inserted 31 movies into tmdb
Inserted 32 movies into tmdb
Insert

Inserted 276 movies into tmdb
Inserted 277 movies into tmdb
Inserted 278 movies into tmdb
Inserted 279 movies into tmdb
Inserted 280 movies into tmdb
Inserted 281 movies into tmdb
Inserted 282 movies into tmdb
Inserted 283 movies into tmdb
Inserted 284 movies into tmdb
Inserted 285 movies into tmdb
Inserted 286 movies into tmdb
Inserted 287 movies into tmdb
Inserted 288 movies into tmdb
Inserted 289 movies into tmdb
Inserted 290 movies into tmdb
Inserted 291 movies into tmdb
Inserted 292 movies into tmdb
Inserted 293 movies into tmdb
Inserted 294 movies into tmdb
Inserted 295 movies into tmdb
Inserted 296 movies into tmdb
Inserted 297 movies into tmdb
Inserted 298 movies into tmdb
Inserted 299 movies into tmdb
Inserted 300 movies into tmdb
Inserted 301 movies into tmdb
Inserted 302 movies into tmdb
Inserted 303 movies into tmdb
Inserted 304 movies into tmdb
Inserted 305 movies into tmdb
Inserted 306 movies into tmdb
Inserted 307 movies into tmdb
Inserted 308 movies into tmdb
Inserted 3

Inserted 550 movies into tmdb
Inserted 551 movies into tmdb
Inserted 552 movies into tmdb
Inserted 553 movies into tmdb
Inserted 554 movies into tmdb
Inserted 555 movies into tmdb
Inserted 556 movies into tmdb
Inserted 557 movies into tmdb
Inserted 558 movies into tmdb
Inserted 559 movies into tmdb
Inserted 560 movies into tmdb
Inserted 561 movies into tmdb
Inserted 562 movies into tmdb
Inserted 563 movies into tmdb
Inserted 564 movies into tmdb
Inserted 565 movies into tmdb
Inserted 566 movies into tmdb
Inserted 567 movies into tmdb
Inserted 568 movies into tmdb
Inserted 569 movies into tmdb
Inserted 570 movies into tmdb
Inserted 571 movies into tmdb
Inserted 572 movies into tmdb
Inserted 573 movies into tmdb
Inserted 574 movies into tmdb
Inserted 575 movies into tmdb
Inserted 576 movies into tmdb
Inserted 577 movies into tmdb
Inserted 578 movies into tmdb
Inserted 579 movies into tmdb
Inserted 580 movies into tmdb
Inserted 581 movies into tmdb
Inserted 582 movies into tmdb
Inserted 5

Inserted 824 movies into tmdb
Inserted 825 movies into tmdb
Inserted 826 movies into tmdb
Inserted 827 movies into tmdb
Inserted 828 movies into tmdb
Inserted 829 movies into tmdb
Inserted 830 movies into tmdb
Inserted 831 movies into tmdb
Inserted 832 movies into tmdb
Inserted 833 movies into tmdb
Inserted 834 movies into tmdb
Inserted 835 movies into tmdb
Inserted 836 movies into tmdb
Inserted 837 movies into tmdb
Inserted 838 movies into tmdb
Inserted 839 movies into tmdb
Inserted 840 movies into tmdb
Inserted 841 movies into tmdb
Inserted 842 movies into tmdb
Inserted 843 movies into tmdb
Inserted 844 movies into tmdb
Inserted 845 movies into tmdb
Inserted 846 movies into tmdb
Inserted 847 movies into tmdb
Inserted 848 movies into tmdb
Inserted 849 movies into tmdb
Inserted 850 movies into tmdb
Inserted 851 movies into tmdb
Inserted 852 movies into tmdb
Inserted 853 movies into tmdb
Inserted 854 movies into tmdb
Inserted 855 movies into tmdb
Inserted 856 movies into tmdb
Inserted 8

KeyError: 'id'